In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import ChebConv
from torch_geometric.utils import dropout_edge
import pickle
from sklearn import metrics

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = torch.load('../data/pan/string_850/data_67_0.0001_2.0.pkl')
data = data.type(torch.FloatTensor).to(device)

data2 = torch.load('../data/pan/string_850/data_67_0.0001_go_2.0.pkl')
data2 = data2.type(torch.FloatTensor).to(device)

data3 = torch.load('../data/pan/string_850/data_67_0.0001_path_2.0.pkl')
data3 = data3.type(torch.FloatTensor).to(device)

Y_data = torch.load('../data/pan/string_850/Y_796_data_2.0.pkl')
Y = torch.tensor(np.logical_or(Y_data.y, Y_data.y_te)).type(torch.FloatTensor).to(device)

with open("../data/pan/string_850/k_sets_novel_2.0.pkl", 'rb') as handle:
    k_sets = pickle.load(handle)
    
e_data = torch.load("../data/pan/string_850/mut_data_miRNA_sub_du_go_path_2.0.pkl")
e_edge_index =e_data.edge_index
e_edge_index = e_edge_index.to(device)

#transformer layer 1 parameter settings
d_model1 = 67  #dimension of input features
d_ff1 = 268  #dimension mapped by feedforward neural network
d_k1 = d_v1 = 67  # dimension of K(=Q), V

#transformer layer 2 parameter settings
d_model2 = 300  #dimension of input features
d_ff2 = 1200  #dimension mapped by feedforward neural network
d_k2 = d_v2 = 300  # dimension of K(=Q), V

n_heads = 6  #number of heads in Multi-Head Attention

/tmp/ipykernel_1246/699151247.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(np.logical_or(Y_data.y, Y_data.y_te)).type(torch.FloatTensor).to(device)


In [3]:
#the framwork of Transformer layer 1
class ScaledDotProductAttention1(nn.Module):
    #use a single attention head to aggregate information
    def __init__(self):
        super(ScaledDotProductAttention1, self).__init__()
 
    def forward(self, Q, K, V):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k1)
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context

class MultiHeadAttention1(nn.Module):
    #use the multi-head attention mechanism to calculate the feature representations 
    def __init__(self):
        super(MultiHeadAttention1, self).__init__()
        self.W_Q = nn.Linear(d_model1, d_k1 * n_heads) 
        self.W_K = nn.Linear(d_model1, d_k1 * n_heads)
        self.W_V = nn.Linear(d_model1, d_v1 * n_heads)
        self.linear = nn.Linear(n_heads * d_v1, d_model1)
 
    def forward(self, H):
        residual, batch_size = H, H.size(0)
        q_s = self.W_Q(H).view(batch_size, -1, n_heads, d_k1).transpose(1,2)
        k_s = self.W_K(H).view(batch_size, -1, n_heads, d_k1).transpose(1,2)
        v_s = self.W_V(H).view(batch_size, -1, n_heads, d_v1).transpose(1,2)
 
        context = ScaledDotProductAttention1()(q_s, k_s, v_s)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v1)
        output = self.linear(context)
        return torch.relu(output + residual) #residual connection
    
class PoswiseFeedForwardNet1(nn.Module):
    #the feedforward neural network of transformer layer 1
    def __init__(self):
        super(PoswiseFeedForwardNet1, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model1, d_ff1, bias=False),
            nn.ELU(),
            nn.Linear(d_ff1, 300, bias=False))
        
    def forward(self, inputs):
        output = self.fc(inputs)
        return output

class TransformerLayer1(nn.Module):
    def __init__(self):
        super(TransformerLayer1, self).__init__()
        self.enc_self_attn = MultiHeadAttention1()
        self.pos_ffn = PoswiseFeedForwardNet1()
 
    def forward(self, enc_inputs):
        enc_outputs = self.enc_self_attn(enc_inputs)
        enc_outputs = self.pos_ffn(enc_outputs)
        return enc_outputs

class TL1(nn.Module):
    def __init__(self):
        super(TL1, self).__init__()
        self.trans = TransformerLayer1()
        
    def forward(self,inputs):
        trans_inputs = inputs.unsqueeze(1)
        trans_outputs = self.trans(trans_inputs)
        outputs = trans_outputs.view(10743,-1)
        return outputs

In [4]:
#the framwork of Transformer layer 1
class ScaledDotProductAttention1(nn.Module):
    #use a single attention head to aggregate information
    def __init__(self):
        super(ScaledDotProductAttention1, self).__init__()
 
    def forward(self, Q, K, V):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k1)
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context

class MultiHeadAttention1(nn.Module):
    #use the multi-head attention mechanism to calculate the feature representations 
    def __init__(self):
        super(MultiHeadAttention1, self).__init__()
        self.W_Q = nn.Linear(d_model1, d_k1 * n_heads) 
        self.W_K = nn.Linear(d_model1, d_k1 * n_heads)
        self.W_V = nn.Linear(d_model1, d_v1 * n_heads)
        self.linear = nn.Linear(n_heads * d_v1, d_model1)
 
    def forward(self, H):
        residual, batch_size = H, H.size(0)
        q_s = self.W_Q(H).view(batch_size, -1, n_heads, d_k1).transpose(1,2)
        k_s = self.W_K(H).view(batch_size, -1, n_heads, d_k1).transpose(1,2)
        v_s = self.W_V(H).view(batch_size, -1, n_heads, d_v1).transpose(1,2)
 
        context = ScaledDotProductAttention1()(q_s, k_s, v_s)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v1)
        output = self.linear(context)
        return torch.relu(output + residual) #residual connection
    
class PoswiseFeedForwardNet1(nn.Module):
    #the feedforward neural network of transformer layer 1
    def __init__(self):
        super(PoswiseFeedForwardNet1, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model1, d_ff1, bias=False),
            nn.ELU(),
            nn.Linear(d_ff1, 300, bias=False))
        
    def forward(self, inputs):
        output = self.fc(inputs)
        return output

class TransformerLayer1(nn.Module):
    def __init__(self):
        super(TransformerLayer1, self).__init__()
        self.enc_self_attn = MultiHeadAttention1()
        self.pos_ffn = PoswiseFeedForwardNet1()
 
    def forward(self, enc_inputs):
        enc_outputs = self.enc_self_attn(enc_inputs)
        enc_outputs = self.pos_ffn(enc_outputs)
        return enc_outputs

class TL1(nn.Module):
    def __init__(self):
        super(TL1, self).__init__()
        self.trans = TransformerLayer1()
        
    def forward(self,inputs):
        trans_inputs = inputs.unsqueeze(1)
        trans_outputs = self.trans(trans_inputs)
        outputs = trans_outputs.view(10743,-1)
        return outputs

In [5]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.T1 = TL1()
        self.T2 = TL2()
        self.GNN = ChebConv(100, 1, K=2, normalization="sym")

        self.lin11 = Linear(67, 300)
        self.lin21 = Linear(67, 300)
        self.lin31 = Linear(67, 300)
        
    def forward(self):
        edge_index, _ = dropout_edge(e_edge_index, p=0.5,force_undirected=True,training=self.training)
        x01 = F.dropout(data, training=self.training)
        x02 = F.dropout(data2, training=self.training)
        x03 = F.dropout(data3, training=self.training)
        
        #learn genes feature representations across various gene association networks by the transformer module
        #apply weight sharing across the two Transformer layers
        x1 = self.T1(x01)
        x2 = torch.relu(x1 + self.lin11(x01)) #residual enhanced activation
        x3 = F.dropout(x2, training=self.training)
        x4 = self.T2(x3)
        
        x6 = self.T1(x02)
        x7 = torch.relu(x6 + self.lin21(x02)) #residual enhanced activation
        x8 = F.dropout(x7, training=self.training)
        x9 = self.T2(x8)
        
        x11 = self.T1(x03)
        x12 = torch.relu(x11 + self.lin31(x03)) #residual enhanced activation
        x13 = F.dropout(x12, training=self.training)
        x14 = self.T2(x13)
        
        #aggregate the three feature representation matrices and obtain predicted results by Chebyshev GCN
        x19 = x4 + x9 + x14
        x20 = F.dropout(x19, training=self.training)
        x21 = self.GNN(x20, edge_index)
        
        return x21

In [6]:
def train(mask):
    model.train()
    optimizer.zero_grad()
    pred = model()
    loss = F.binary_cross_entropy_with_logits(pred[mask], Y[mask])
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

@torch.no_grad()
def test(mask):
    model.eval()
    x = model()
    pred = torch.sigmoid(x[mask]).cpu().detach().numpy()
    return pred

In [ ]:
epochs = 550
time_start = time.time()

tr_mask = k_sets[0]
te_mask = k_sets[1]
model = net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
for t in range(epochs):
    train(tr_mask)   
pred = test(te_mask) 

print(time.time() - time_start)
np.savetxt("../data/pan/string_850/pred_novel_2.0.txt",pred,fmt="%.10e",delimiter=" ")